In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
#import seaborn as sns
import pymongo
from pprint import pprint

In [3]:
client = pymongo.MongoClient()
db = client.metrics

In [4]:
def group_by(df, bycols, agg_map):
    """

    @param df:      DataFrame
    @param bycols:  str or list
                        Column(s) to group by
    @param agg_map: dictionary or list of 2-tuples
                        Mapping from column to aggregate function e.g. [("city", "count"), ("salary", "mean"]
    @return:        DataFrame
                        Flattened dataframe, with multi-level index removed
    """
    grps = []
    if type(bycols) == str:
        bycols = [bycols]

    if type(agg_map) == dict:
        agg_map = agg_map.items()

    for k,v in agg_map:
        grp = df[bycols + [k]].groupby(bycols, ).agg(v)
        grp.reset_index(inplace=True)
        grp["%s(%s)" % (v,k)] = grp[k]
        del grp[k]
        grps.append(grp)

    m = grps[0]
    for grp in grps[1:]:
        m = pd.merge(m, grp, on=bycols, how="inner")
    return m

In [5]:
from bson.son import SON # needed to ensure dictionary is ordered (python default is not)
import hashlib

def hash_feats(fts):
    vals = fts.values
    joined = "|".join(map(lambda s: str(s),vals)).encode('utf-8') 
    return hashlib.sha224(joined).hexdigest()

def get_df_sorted_by_f1score(collection, params=None, filter_cols=True):
    if not params:
        params = []
    if type(params) == str:
        params = params.split(",")
    
    project = {
            "weighted_f1_score":"$WEIGHTED_MEAN_CONCEPT_CODES.f1_score",
            "micro_f1_score":  "$MICRO_F1.f1_score",
            "micro_recall":    "$MICRO_F1.recall",
            "micro_precision": "$MICRO_F1.precision",
    
    # PARAMETERS            
            "window_size":    "$parameters.window_size",
            "feats":          "$parameters.extractors",
            "count": {        "$size" : "$parameters.extractors" },
            "asof" :          "$asof",
            "_id":1
    }
    
    # No count for HMM
    if "_hmm" in collection.lower():
        del project["count"]
    
    for param in params:
        project[param] = "$parameters." + param

    feats_pipeline = [{
        "$project": project
    },
    {
        "$match":{
            "micro_f1_score": { "$exists" : True }        
        }
    },
    {
        "$sort":{
            "micro_f1_score": -1
        }
    },
    ]
    
    rows = [row for row in db[collection].aggregate(feats_pipeline)]
    df = pd.DataFrame(rows).sort_values("micro_f1_score", ascending=False)
    if params:
        df["hs_params"] = df[params].apply(hash_feats, axis=1)
        
    if filter_cols:
        cols = ["micro_f1_score", "micro_recall" ,"micro_precision" ] + params
        return df[cols]
    return df

In [70]:
from Metrics import rpf1a_from_tp_fp_tn_fn
from collections import defaultdict

def tally_counts(r, filter):
    tally = defaultdict(int)
    for k,v in r.items():
        if filter(k):
            for prop in "tp,tn,fp,fn".split(","):
                tally[prop] += v[prop]
    return tally

def get_causal_relation_metrics(collection, params):
    dicts = []
    for r in db[collection].find({}):
        d = {}
        cr_counts = tally_counts(r, lambda c: "->" in c)
        (rec, p, cr_f1, a) = rpf1a_from_tp_fp_tn_fn(cr_counts["tp"],cr_counts["fp"],cr_counts["tn"],cr_counts["fn"])
        d["cr_micro_f1"] = cr_f1
        d["cr_micro_rec"]  = rec
        d["cr_micro_prec"] = p
        concept_counts = tally_counts(r, lambda c: c[0].isdigit())
        (rec, p, concept_f1, a) = rpf1a_from_tp_fp_tn_fn(concept_counts["tp"],concept_counts["fp"],concept_counts["tn"],concept_counts["fn"])
        d["concept_micro_f1"] = concept_f1
        d["concept_micro_rec"]  = rec
        d["concept_micro_prec"] = p
        parms = r["parameters"]
        for p in params:
            d[p] = parms[p]
        dicts.append(d)
    df = pd.DataFrame(dicts)
    fields = ("cr_micro_f1,cr_micro_rec,cr_micro_prec,concept_micro_f1,concept_micro_rec,concept_micro_prec," + ",".join(params)).split(",")
    return df[fields].sort_values("cr_micro_f1", ascending=False)

In [71]:
def round_data(df, places=3):
    df_copy = df.copy()
    fmt_str = "{0:." + str(places) + "f}"
    cols = set([v for v in df_copy.columns.values if "micro_" in v])
    for c in cols:
        df_copy[c] = df[c].apply(lambda d: fmt_str.format(d))  
    return df_copy

## Coral Bleaching

In [87]:
params = "merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size"
df = get_causal_relation_metrics("CR_CB_TAGGING_TD_RNN", params.split(","))
df

,cr_micro_f1,cr_micro_rec,cr_micro_prec,concept_micro_f1,concept_micro_rec,concept_micro_prec,merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size
4,0.324812,0.821033,0.202452,0.618733,0.892872,0.473388,sum,2,True,True,128
5,0.316805,0.828690,0.195836,0.627167,0.866187,0.491532,sum,2,True,True,256
3,0.306534,0.781365,0.190667,0.630436,0.897121,0.485972,sum,2,True,True,64
2,0.294755,0.818911,0.179722,0.582642,0.907268,0.429105,sum,1,True,True,256
1,0.292087,0.814852,0.177934,0.595468,0.907900,0.443015,sum,1,True,True,128
0,0.288426,0.783672,0.176736,0.594902,0.929284,0.437483,sum,1,True,True,64


In [88]:
params = "merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size"
df = get_causal_relation_metrics("CR_CB_TAGGING_VD_RNN", params.split(","))
df

,cr_micro_f1,cr_micro_rec,cr_micro_prec,concept_micro_f1,concept_micro_rec,concept_micro_prec,merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size
4,0.309565,0.778443,0.193197,0.614124,0.886260,0.469850,sum,2,True,True,128
5,0.299797,0.787092,0.185162,0.626196,0.861336,0.491908,sum,2,True,True,256
3,0.294635,0.747172,0.183497,0.628916,0.896508,0.484347,sum,2,True,True,64
2,0.285714,0.793081,0.174244,0.578017,0.897520,0.426271,sum,1,True,True,256
1,0.281959,0.777778,0.172190,0.590700,0.900810,0.439425,sum,1,True,True,128
0,0.278671,0.757818,0.170726,0.594726,0.925987,0.438027,sum,1,True,True,64


In [90]:
params = "merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size"
df = get_causal_relation_metrics("CR_SC_TAGGING_VD_RNN", params.split(","))
df

,cr_micro_f1,cr_micro_rec,cr_micro_prec,concept_micro_f1,concept_micro_rec,concept_micro_prec,merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size
3,0.391120,0.825292,0.256290,0.622329,0.772546,0.521020,sum,2,True,True,64
4,0.389848,0.835344,0.254253,0.637635,0.803492,0.528535,sum,2,True,True,128
2,0.363051,0.828106,0.232489,0.624034,0.776525,0.521604,sum,1,True,True,256
1,0.344212,0.805790,0.218849,0.629691,0.814987,0.513045,sum,1,True,True,128
0,0.341861,0.805991,0.216937,0.644597,0.822060,0.530150,sum,1,True,True,64
